# Generic

Helpers. All copied from previous notebooks

In [1]:
import os
import math
import base64
import binascii
import numpy as np
import matplotlib.pyplot as plt

def to_bytes(d, format):
    if isinstance(d, (bytes, bytearray)):
        return d
    elif format == 'hex':
        return bytes(bytearray.fromhex(d))
    elif format == 'base64':
        return base64.b64decode(d)
    elif format == 'str' or format == 'bytes':
        return d.encode()
    elif format == 'int':
        return to_bytes(hex(d)[2:], 'hex')

def bytes_to(b, format):
    if not isinstance(b, (bytes, bytearray)):
        return b
    elif format == 'hex':
        return binascii.hexlify(b).decode()
    elif format == 'base64':
        return base64.b64encode(b).decode()
    elif format == 'str':
        return b.decode()
    elif format == 'bytes':
        return b
    elif format == 'int':
        return int(bytes_to(b, 'hex')[2:], 16)
    
def draw_bytestring(bs):
    """Represent a byte string as a matrix of color
    """
    tmp = bs + bytes(math.ceil(math.sqrt(len(bs)))**2 - len(bs))
    tmp = [int(b) for b in tmp]
    m = [[tmp[i:i+int(math.sqrt(len(tmp)))] for i in range(0, len(tmp), int(math.sqrt(len(tmp))))]]
    matrix = np.matrix(m[0])
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.set_aspect('equal')
    plt.imshow(matrix, interpolation='nearest', cmap=plt.cm.binary)
    plt.colorbar()
    plt.show()

# Crypto

Simple crypto functions that are only accessory. Copied from previous notebooks.

In [2]:
import itertools
from Crypto.Cipher import AES

def pad(text, blocksize=16):
    padsize = blocksize - (len(text) % blocksize)
    if padsize == 0:
        padsize = blocksize
    return text + bytes([padsize]*padsize)
assert pad(b"YELLOW SUBMARINE", blocksize=20) == b'YELLOW SUBMARINE\x04\x04\x04\x04'
def unpad(text):
    """Validate padding and return unpadded value"""
    assert text[-text[-1]:] == bytes([text[-1]]*text[-1])
    return text[:-text[-1]]
assert unpad(pad(b"this is random text", blocksize=7)) == b"this is random text"

def xor(d1, d2, format):
    b1,b2 = to_bytes(d1, format), to_bytes(d2, format)
    return bytes_to(bytes(char1 ^ char2 for char1,char2 in zip(b1,b2)), format)

def decrypt_aes_ecb(cipher, key, IV=None, use_padding=True):
    """No magic here
    (The IV is not actually used)"""
    aes = AES.new(key, AES.MODE_ECB)
    if use_padding:
        return unpad(aes.decrypt(cipher))
    else:
        return aes.decrypt(cipher)
def encrypt_aes_ecb(clear, key, IV=None, use_padding=True):
    """No magic here
    (The IV is not actually used)"""
    aes = AES.new(key, AES.MODE_ECB)
    if use_padding:
        clear = pad(clear)
    return aes.encrypt(clear)
assert decrypt_aes_ecb(encrypt_aes_ecb(b'test', 'YELLOW SUBMARINE'), 'YELLOW SUBMARINE') == b'test'


def encrypt_aes_cbc(clear, key, IV=None, debug=False):
    """CBC using the ECB mode of above
    """
    if IV is None:
        IV=b'\x00'*16
    clear = pad(clear, blocksize=16)
    prev_block = IV
    cipher = b''
    for blockstart in range(0, len(clear), 16):
        block = clear[blockstart:blockstart+16]
        if debug:
            print("encrypting...", block)
        cipher_block = encrypt_aes_ecb(xor(prev_block, block, 'bytes'), key, use_padding=False)
        prev_block = cipher_block
        cipher += cipher_block
    if debug:
        print("resulting cipher :", cipher)
    return cipher
def decrypt_aes_cbc(cipher, key, IV=None, debug=False):
    """CBC using the ECB mode of above
    """
    if IV is None:
        IV=b'\x00'*16
    clear = b''
    prev_block = IV
    for blockstart in range(0, len(cipher), 16):
        block = cipher[blockstart:blockstart+16]
        if debug:
            print("decrypting...", block)
        tmp = decrypt_aes_ecb(block, key, use_padding=False)
        clear_block = xor(prev_block, tmp, 'bytes')
        prev_block = cipher[blockstart:blockstart+16]
        clear += clear_block
    if debug:
        print("resulting clear :", clear)
    return unpad(clear)

test_iv = os.urandom(16)
assert decrypt_aes_cbc(encrypt_aes_cbc(b"cookingMCslikeapoundofbacon", b'YELLOW SUBMARINE', IV=test_iv), b'YELLOW SUBMARINE', IV=test_iv) == b"cookingMCslikeapoundofbacon"

    

In [3]:
pad(b"cookingMCslikeapoundofbacon", blocksize=16)

b'cookingMCslikeapoundofbacon\x05\x05\x05\x05\x05'

# Cryptopals Notebook

## Set 5

### Exercice 33

Diffie-Hellman.

In [4]:
import random
import hashlib

class DH:
    
    def __init__(self, p=37, g=5, a=None):
        self._p, self._g = p, g
        self._A, self._B, self._s = None, None, None
        self.generate_private_key(a)
        
    def generate_private_key(self, a=None):
        self._a = random.randint(0, self._p-1)
        if a is not None:
            self._a = a
        # goooo, python! power-mod is included
        # otherwise - not that hard to implement :
        # use a classic power algorithm but mod
        # the result everytime you multiply by the base
        self._A = pow(self._g, self._a, self._p)
        
    @property
    def public_key(self):
        if self._A is None:
            self.generate_private_key()
        return self._A

    def generate_session_key(self, B):
        # Generate the session key from the distant public key
        # and the local private key
        self._B = B
        self._s = pow(B, self._a, self._p)
        return self._s
        
    @property
    def session_key(self):
        return self._s
    
    @property
    def key(self):
        sha1 = hashlib.sha1()
        sha1.update(hex(self._s)[2:].encode())
        return sha1.digest()

In [5]:
alice = DH()
bob = DH()
alice.generate_session_key(bob.public_key)
bob.generate_session_key(alice.public_key)
print('Alice session key :', alice.session_key)
print('Bob session key :', bob.session_key)
assert alice.session_key == bob.session_key

Alice session key : 36
Bob session key : 36


In [6]:
class NISTDH(DH):
    def __init__(self, a=None):
        self._p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
        self._g = 2
        super().__init__(self._p, self._g, a)

In [7]:
alice = NISTDH()
bob = NISTDH()
alice.generate_session_key(bob.public_key)
bob.generate_session_key(alice.public_key)
print('Alice session key :', alice.key)
print('Bob session key :', bob.key)
assert alice.session_key == bob.session_key

Alice session key : b'K\xc4n\xbdGX>8\x1b\xac\xb4\xb4\xe72\xd4x\xa5`\x8e\xe5'
Bob session key : b'K\xc4n\xbdGX>8\x1b\xac\xb4\xb4\xe72\xd4x\xa5`\x8e\xe5'


### Exercice 34

MITM on DH. Let the fun begin ! We will emulate network actors with a simple message passing class. Back and forth is handled using callbacks and predefined functions. Messages are arbitrary python objects/

In [8]:
class NetworkActor:
    
    def __init__(self, name, actions):
        self._log = []
        self._name = name
        self._actions = actions
        self._dh = NISTDH()
        self._action_idx = 0
        
    @property
    def name(self):
        return self._name
    
    def initiate(self, receiver):
        # Trigger first action
        action = self._actions[self._action_idx]
        self._action_idx = (self._action_idx + 1) % len(self._actions)
        action(self, receiver)
    
    def send(self, receiver, msg):
        receiver.receive(self, msg)
        
    def receive(self, sender, msg):
        print('[.]', sender.name, '->', self.name, '\n', msg, '\n\n')
        if self._actions:
            action = self._actions[self._action_idx]
            self._action_idx = (self._action_idx + 1) % len(self._actions)
            action(self, sender, msg)

In [9]:
# First, the sequence without the MITM actor
import os

def ex34_initiate(alice, bob):
    # First, Alice sends its parameters to Bob
    alice._dh = NISTDH()
    bob.receive(alice, {"p": alice._dh._p, "g": alice._dh._g, "A": alice._dh.public_key})

def ex34_m1(bob, alice, msg):
    # Then bob replies with its public key
    p, g, A = msg["p"], msg["g"], msg["A"]
    bob._dh = DH(p, g)
    bob._dh.generate_session_key(A)
    bob.send(alice, {"B": bob._dh.public_key})

def ex34_m2(alice, bob, msg):
    # Alice replies with encrypted random message
    B = msg["B"]
    alice._dh.generate_session_key(B)
    payload = b"cookingMCslikeapoundofbacon"
    iv = os.urandom(16)
    key = alice._dh.key[:16]
    
    # Alice expects the same message in reply, but encrypted with
    # a different IV
    alice.expected_reply = payload
    alice.iv_to_avoid = iv
    
    encrypted_msg = encrypt_aes_cbc(payload, key, IV=iv)
    alice.send(bob, encrypted_msg + iv)
    
def ex34_m3(bob, alice, msg):
    # Bob decrypts Alice's message and re-encrypts it with another IV
    payload = msg[:-16]
    iv = msg[-16:]
    key = bob._dh.key[:16]
    cleartext = decrypt_aes_cbc(payload, key, IV=iv)
    
    new_iv = os.urandom(16) # don't check if it's the same. YOLO !
    encrypted_msg = encrypt_aes_cbc(cleartext, key, IV=new_iv)
    bob.send(alice, encrypted_msg + new_iv)
    
def ex34_final(alice, _, msg):
    # Alice decrypts incoming message and checks that it is the same
    payload = msg[:-16]
    iv = msg[-16:]
    key = alice._dh.key[:16]
    cleartext = decrypt_aes_cbc(payload, key, IV=iv)
    
    assert iv != alice.iv_to_avoid
    assert cleartext == alice.expected_reply
    

Alice = NetworkActor("Alice", actions=[ex34_initiate, ex34_m2, ex34_final])
Bob = NetworkActor("Bob", actions=[ex34_m1, ex34_m3])

Alice.initiate(Bob)

[.] Alice -> Bob 
 {'p': 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919, 'g': 2, 'A': 388712358194302795818422031719832238594049407398319062885254342311945502195044750945184834394649615937633754440978528453518137165115881724920167032850623966116635833295822452294389546620408370282213867342900933773052631051863236898541496510939515210326452331618952173448196502251924613045991916948801099560601081866748733072210092528528461349543677391147605391209900959096967088700833962576835556329098841968585577355312438016896290942604047810078803971878356146} 


[.] Bob -> Alice 
 {'B': 20961

$$For\ B:\\\begin{eqnarray}
s &=& (A ^ b)\mod p\\
&=& (p ^ b)\mod p\\
&=& 0\\\end{eqnarray}$$

$$For \ A:\\\begin{eqnarray}
s &=& (B ^ a)\mod p\\
&=& (p ^ a)\mod p\\
&=& 0\end{eqnarray}$$

In [10]:
# Now, the sequence with the attacker (Eve)

class MITM(NetworkActor):
    def __init__(self, alice, bob, name, actions):
        self.alice = alice
        self.bob = bob
        super().__init__(name, actions)

def ex34_initiate_i(eve, _, msg):
    # Eve alters the message !
    # Bob still should think the message came from Alice in a real world attack
    eve.intercepted_p, eve.intercepted_g, eve.intercepted_A = msg["p"], msg["g"], msg["A"]
    eve.bob.receive(eve, {"p": msg["p"], "g": msg["g"], "A": msg["p"]})
    
    
def ex34_m1_i(eve, _, msg):
    # Eve also modifies this one
    eve.intercepted_B = msg["B"]
    eve.alice.receive(eve, {"B": eve.intercepted_p})
    
def ex34_m2_i(eve, _, msg):
    # Eve intercepts the message and decrypts it easily
    sha1 = hashlib.sha1()
    sha1.update(hex(0)[2:].encode())
    key = sha1.digest()[:16]
    payload = msg[:-16]
    iv = msg[-16:]
    cleartext = decrypt_aes_cbc(payload, key, IV=iv)
    eve.intercepted_encrypted_message_1 = msg
    eve.cleartext_1 = cleartext
    eve.bob.receive(eve, msg)
    
def ex34_m3_i(eve, _, msg):
    # confirm with the return message
    sha1 = hashlib.sha1()
    sha1.update(hex(0)[2:].encode())
    key = sha1.digest()[:16]
    payload = msg[:-16]
    iv = msg[-16:]
    cleartext = decrypt_aes_cbc(payload, key, IV=iv)
    eve.intercepted_encrypted_message_2 = msg
    eve.cleartext_2 = cleartext
    assert eve.cleartext_1 == eve.cleartext_2
    print('Intercepted message', cleartext, "\n\n")
    
    eve.alice.receive(eve, msg)
    

Eve = MITM(Alice, Bob, "Eve", actions=[ex34_initiate_i, ex34_m1_i, ex34_m2_i, ex34_m3_i])

Alice.initiate(Eve)

[.] Alice -> Eve 
 {'p': 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919, 'g': 2, 'A': 1329431685833625354369860065784686073965688617510080613112570227428883292768243821791969074774511404632627311404921715803091588370174584077370629645943689855392206344207534741356889114780022431234497003466903405672350843803636740453906242964325222247968752638766976744916122197395140664099270304807661421953852513165863491773245427840445191693158262002642699059982047521075151710540658887488492526742378643409150867642221417551975343503506261965661753573500902247} 


[.] Eve -> Bob 
 {'p': 241031

### Exercice 35

Same idea but with manipulating g instead of the public keys.

In [11]:
# The base-case without the MITM

def ex35_initiate(alice, bob):
    # All messages now go through Eve
    alice._dh = NISTDH()
    alice._sent_p, alice._sent_g = alice._dh._p, alice._dh._g
    bob.receive(alice, {"p": alice._dh._p, "g": alice._dh._g})
    
def ex35_m1(bob, alice, msg):
    # Respond with ACK
    p, g = msg["p"], msg["g"]
    bob._dh = DH(p, g)
    alice.receive(bob, {"g": g, "ACK": "ACK"})
    
def ex35_m2(alice, bob, msg):
    assert msg["ACK"] == "ACK"
    
    # The exercice is worded pretty loosely for this one.
    # We just assume that we can change the g value of both parties
    # else there is a key mismatch in the end. If we don't assume
    # this, we could always renegotiate public keys with Alice for
    # the MITM attack, but that kinda defeats the purpose of giving
    # Bob a manipulated g value.
    alice._dh = DH(alice._sent_p, msg["g"])
    alice._dh.generate_private_key()
    bob.receive(alice, alice._dh.public_key)
    
def ex35_m3(bob, alice, msg):
    bob._dh.generate_session_key(msg)
    alice.receive(bob, bob._dh.public_key)

def ex35_m4(alice, bob, msg):
    # Alice replies with encrypted random message
    B = msg
    alice._dh.generate_session_key(B)
    payload = b"cookingMCslikeapoundofbacon"
    iv = os.urandom(16)
    key = alice._dh.key[:16]
    
    # Alice expects the same message in reply, but encrypted with
    # a different IV
    alice.expected_reply = payload
    alice.iv_to_avoid = iv
    
    encrypted_msg = encrypt_aes_cbc(payload, key, IV=iv)
    alice.send(bob, encrypted_msg + iv)
    
def ex35_m5(bob, alice, msg):
    # Bob decrypts Alice's message and re-encrypts it with another IV
    payload = msg[:-16]
    iv = msg[-16:]
    key = bob._dh.key[:16]
    cleartext = decrypt_aes_cbc(payload, key, IV=iv)
    
    new_iv = os.urandom(16) # don't check if it's the same. YOLO !
    encrypted_msg = encrypt_aes_cbc(cleartext, key, IV=new_iv)
    bob.send(alice, encrypted_msg + new_iv)
    
def ex35_final(alice, _, msg):
    # Alice decrypts incoming message and checks that it is the same
    payload = msg[:-16]
    iv = msg[-16:]
    key = alice._dh.key[:16]
    cleartext = decrypt_aes_cbc(payload, key, IV=iv)
    
    assert iv != alice.iv_to_avoid
    assert cleartext == alice.expected_reply

Alice = NetworkActor("Alice", actions=[ex35_initiate, ex35_m2, ex35_m4, ex35_final])
Bob = NetworkActor("Bob", actions=[ex35_m1, ex35_m3, ex35_m5])

Alice.initiate(Bob)

[.] Alice -> Bob 
 {'p': 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919, 'g': 2} 


[.] Bob -> Alice 
 {'g': 2, 'ACK': 'ACK'} 


[.] Alice -> Bob 
 882665450804064424646685508158717852644913876155024618774923198413315030318799501392300124829973480213692044148262688802232547730839980925399387985420951877605418323334856951605356657876919021038296078438267962224503885036535336925212196930164764238118714950145970595896267231648355458274042288318944559434780001474185118426370772446084987000464796734991166378004167655868633550445677823012986422804539226681548599502665286921175825032

All the different scenarios :

$$With\ g=1:\\\begin{eqnarray}
A &=& (1 ^ a)\mod p\\
&=& 1\\
\implies s &=& (1 ^ a)\mod p\\
&=& 1\\\end{eqnarray}$$

$$With\ g=p:\\\begin{eqnarray}
A &=& (p ^ a)\mod p\\
&=& 0\\
\implies s &=& (0 ^ a)\mod p\\
&=& 0\\\end{eqnarray}$$

$$With\ g=p-1:\\\begin{eqnarray}
A &=& ((p-1) ^ a)\mod p\\
&=& 1\\
\implies s &=& (1 ^ a)\mod p\\
&=& 1\\\end{eqnarray}$$

In [12]:
# Aaaand with the MITM
    
def ex35_initiate_i(eve, _, msg):
    # Eve intercepts this message and changes the g value
    eve.intercepted_p, eve.intercepted_g = msg["p"], msg["g"]
    fake_msg = {"p": msg["p"], "g": eve.fake_g}
    eve.bob.receive(eve, fake_msg)
    
def passthrough_bob_to_alice(eve, _, msg):
    # Generic passthrough method
    print("generic b>a")
    eve.alice.receive(eve, msg)
    
def passthrough_alice_to_bob(eve, _, msg):
    # Generic passthrough method
    print("generic a>b")
    eve.bob.receive(eve, msg)
    
def ex35_m4_i(eve, _, msg):
    # Eve gets the encrypted message, but already knows the key
    sha1 = hashlib.sha1()
    sha1.update(hex(eve.forced_key)[2:].encode())
    key = sha1.digest()[:16]
    payload = msg[:-16]
    iv = msg[-16:]
    print("key", key)
    cleartext = decrypt_aes_cbc(payload, key, IV=iv)
    eve.intercepted_encrypted_message_1 = msg
    eve.cleartext_1 = cleartext
    eve.bob.receive(eve, msg)
    
def ex35_m5_i(eve, _, msg):
    # confirm with the return message
    sha1 = hashlib.sha1()
    sha1.update(hex(eve.forced_key)[2:].encode())
    key = sha1.digest()[:16]
    payload = msg[:-16]
    iv = msg[-16:]
    cleartext = decrypt_aes_cbc(payload, key, IV=iv)
    eve.intercepted_encrypted_message_2 = msg
    eve.cleartext_2 = cleartext
    assert eve.cleartext_1 == eve.cleartext_2
    print('Intercepted message', cleartext)
    
    eve.alice.receive(eve, msg)
    
Eve = MITM(Alice, Bob, "Eve", actions=[ex35_initiate_i, passthrough_bob_to_alice,
                                       passthrough_alice_to_bob,
                                       passthrough_bob_to_alice,
                                       ex35_m4_i, ex35_m5_i])


In [13]:
p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff

# g = 1
Eve.fake_g, Eve.forced_key = 1, 1
Alice.initiate(Eve)

# g = p - 1
Eve.fake_g, Eve.forced_key = p-1, 1
Alice.initiate(Eve)

# g = p
Eve.fake_g, Eve.forced_key = p, 0
Alice.initiate(Eve)

[.] Alice -> Eve 
 {'p': 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919, 'g': 2} 


[.] Eve -> Bob 
 {'p': 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919, 'g': 1} 




### Exercice 36

Implement SRP

In [14]:
import os
import random

BIGP = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
MEDIANP = 107
SMALLP = 7

def sha256(msg):
    hasher = hashlib.sha256()
    hasher.update(msg.encode())
    return hasher.digest()

def sha256mac(key, message):
    return sha256(key + message) # Meh. Good enough :p.

In [15]:
# Both client and server are aware of these at one point at least
N, g, k = BIGP, 2, 3
P = "password"

# Server ("account creation" - before the actual auth)
salt = random.randint(0, 2**64)
xH = sha256(hex(salt) + P)
x = bytes_to(xH, "int")
v = pow(g, x, N)
# Save to server what needs to be saved (consider the rest discarded)
server = {'v': v, 'salt': salt}

# Client (start of authentication)
I = "email@test.com"
a = random.randint(0, 2**64)
A = pow(g, a, N)
client = {'a': a, 'A': A}
server['A'] = A # I and A are transmitted 
server['I'] = I

# Server
b = random.randint(0, 2**64)
B = (k*v + pow(g, b, N)) % N
server['b'] = b
server['B'] = B
client['B'] = B # salt and B are transmitted
client['salt'] = salt

# Both
uH = sha256(str(A) + str(B))
u = bytes_to(uH, 'int')
client['u'] = u
server['u'] = u

# Client (knows his password and his session random key a)
xH = sha256(hex(salt) + P)
x = bytes_to(xH, 'int')
client['x'] = x
client['S'] = pow(B - k*pow(g, x, N), a + u*x, N)

# Server
server['S'] = pow(A * pow(v, u, N), b, N)

# Ensure it has worked :
assert server['S'] == client['S']

Why it works :

$$\begin{align*}
S_{client}
&= (B-k.g^x)^{a+u.x} \ \ (\mathrm{mod}\ N) \\
&= (k.v+g^b-k.g^x)^{a+u.x} \ \ (\mathrm{mod}\ N) \\
&= (g^b)^{a+u.x} \ \ (\mathrm{mod}\ N) \\
&= (g^{a+u.x})^b \ \ (\mathrm{mod}\ N) \\
&= (g^a.(g^x)^u)^b \ \ (\mathrm{mod}\ N) \\
&= (A.v^u)^b \ \ (\mathrm{mod}\ N) \\
&= S_{server}
\end{align*}$$

### Exercice 37

Break SRP with a 0-key

In [16]:
# First, let's make SRP into real server and clients

# Global constants
N, g, k = BIGP, 2, 3

class Server:
    def __init__(self):
        self.accounts = {}
        self.sessions = []
        
    
    def create_account(self, I, P):
        print("Creating account for", I)
        salt = random.randint(0, 2**64)
        xH = sha256(hex(salt) + P)
        x = bytes_to(xH, "int")
        v = pow(g, x, N)
        self.accounts[I] = {'v': v, 'salt': salt}
    
    def authenticate(self, I, A, cb):
        print("Starting authentication flow for identity", I)
        salt = self.accounts[I]['salt']
        v = self.accounts[I]['v']
        b = random.randint(0, 2**64)
        B = (k*v + pow(g, b, N)) % N
        uH = sha256(str(A) + str(B))
        u = bytes_to(uH, 'int')
        S = pow(A * pow(self.accounts[I]['v'], u, N), b, N)
        K = bytes_to(sha256(str(S)),'base64')
        HMAC = sha256mac(K, str(salt))
        self.sessions.append({'I': I, 'A': A, 'b': b, 'K': K, 'HMAC': HMAC})
        cb(salt, B, len(self.sessions)-1, self.validate_authentication)
    64
    def validate_authentication(self, session_id, HMAC):
        if self.sessions[session_id]['HMAC'] == HMAC:
            print('Client', self.sessions[session_id]['I'], 'is authenticated !')
            return True
        else:
            print('Client', self.sessions[session_id]['I'], 'failed authentication...')
            return False

    
class Client:
    def __init__(self, identity, username, password):
        self.identity = identity
        self.username = username
        self.password = password
        self.session = {}
        
    def register_account(self, s):
        print(self.identity, "is creating an account for username", self.username)
        s.create_account(self.username, self.password)
    
    def authenticate(self, s):
        print(self.identity, "is trying to authenticate as", self.username)
        a = random.randint(0, 2**64)
        A = pow(g, a, N)
        self.session = {'a': a, 'A':A}
        s.authenticate(self.username, A, self.exchange1)
        
    def exchange1(self, salt, B, session_id, cb):
        self.session['salt'] = salt
        self.session['B'] = B
        uH = sha256(str(self.session['A']) + str(B))
        u = bytes_to(uH, 'int')
        xH = sha256(hex(salt) + self.password)
        x = bytes_to(xH, 'int')
        a = self.session['a']
        S = pow(B - k*pow(g, x, N), a + u*x, N)
        K = bytes_to(sha256(str(S)),'base64')
        HMAC = sha256mac(K, str(self.session['salt']))
        self.session['K'] = K
        self.session['HMAC'] = HMAC
        self.session['session_id'] = session_id
        if cb(session_id, HMAC):
            print(self.identity, "is authenticated as", self.username)
        

Alice = Server()
Bob = Client('Bob', 'bob', 'bob@passw0rd')

Bob.register_account(Alice)
Bob.authenticate(Alice)

Bob is creating an account for username bob
Creating account for bob
Bob is trying to authenticate as bob
Starting authentication flow for identity bob
Client bob is authenticated !
Bob is authenticated as bob


In [17]:
# Now create a malicious client, that does not know the password !
# Eve will use A = 0, which gives S = (0 x v^u)^b = 0
# Given that everything is mod N, A = N, 2*N etc... also works.

class MaliciousClient(Client):
    
    def authenticate(self, s):
        print(self.identity, "is trying to break in as", self.username)
        a = 0
        i = random.randint(0, 100)
        A = N * i
        print("Eve : A is N *", i)
        self.session = {'a': a, 'A':A}
        s.authenticate(self.username, A, self.exchange1)
        
    def exchange1(self, salt, B, session_id, cb):
        self.session['salt'] = salt
        S = 0
        K = bytes_to(sha256(str(S)),'base64')
        HMAC = sha256mac(K, str(self.session['salt']))
        self.session['K'] = K
        self.session['HMAC'] = HMAC
        self.session['session_id'] = session_id
        if cb(session_id, HMAC):
            print(self.identity, "is authenticated as", self.username, "!")
            
Eve = MaliciousClient("Eve", "bob", "")
Eve.authenticate(Alice)

Eve is trying to break in as bob
Eve : A is N * 34
Starting authentication flow for identity bob
Client bob is authenticated !
Eve is authenticated as bob !


### Exercice 38

Implement and break a simplified version of SRP, that uses a less secure B value.

In [18]:
class Server:
    def __init__(self):
        self.accounts = {}
        self.sessions = []
        
    
    def create_account(self, I, P):
        print("Creating account for", I)
        salt = random.randint(0, 2**128)
        xH = sha256(hex(salt) + P)
        x = bytes_to(xH, "int")
        v = pow(g, x, N)
        self.accounts[I] = {'v': v, 'salt': salt}
    
    def authenticate(self, I, A, cb):
        print("Starting authentication flow for identity", I)
        salt = self.accounts[I]['salt']
        v = self.accounts[I]['v']
        b = random.randint(0, 2**128)
        B = pow(g, b, N)
        uH = sha256(str(A) + str(B))
        u = bytes_to(uH, 'int')
        S = pow(A * pow(self.accounts[I]['v'], u, N), b, N)
        K = bytes_to(sha256(str(S)),'base64')
        HMAC = sha256mac(K, str(salt))
        self.sessions.append({'I': I, 'A': A, 'b': b, 'K': K, 'HMAC': HMAC})
        cb(salt, B, len(self.sessions)-1, self.validate_authentication)

    def validate_authentication(self, session_id, HMAC):
        if self.sessions[session_id]['HMAC'] == HMAC:
            print('Client', self.sessions[session_id]['I'], 'is authenticated !')
            return True
        else:
            print('Client', self.sessions[session_id]['I'], 'failed authentication...')
            return False

    
class Client:
    def __init__(self, identity, username, password):
        self.identity = identity
        self.username = username
        self.password = password
        self.session = {}
        
    def register_account(self, s):
        print(self.identity, "is creating an account for username", self.username)
        s.create_account(self.username, self.password)
    
    def authenticate(self, s):
        print(self.identity, "is trying to authenticate as", self.username)
        a = random.randint(0, 2**128)
        A = pow(g, a, N)
        self.session = {'a': a, 'A':A}
        s.authenticate(self.username, A, self.exchange1)
        
    def exchange1(self, salt, B, session_id, cb):
        self.session['salt'] = salt
        self.session['B'] = B
        uH = sha256(str(self.session['A']) + str(B))
        u = bytes_to(uH, 'int')
        xH = sha256(hex(salt) + self.password)
        x = bytes_to(xH, 'int')
        a = self.session['a']
        S = pow(B, a + u*x, N)
        K = bytes_to(sha256(str(S)),'base64')
        HMAC = sha256mac(K, str(self.session['salt']))
        self.session['K'] = K
        self.session['HMAC'] = HMAC
        self.session['session_id'] = session_id
        if cb(session_id, HMAC):
            print(self.identity, "is authenticated as", self.username)
        

In [19]:
Alice = Server()
Bob = Client('Bob', 'bob', 'bob@passw0rd')

Bob.register_account(Alice)
Bob.authenticate(Alice)

Bob is creating an account for username bob
Creating account for bob
Bob is trying to authenticate as bob
Starting authentication flow for identity bob
Client bob is authenticated !
Bob is authenticated as bob


In [20]:
with open('/usr/share/dict/british-english') as _in:
    words = _in.read().split('\n')[:1000]

class MITMServer:
    
    def __init__(self, server, manipulate=False):
        self.accounts = {}
        self.sessions = []
        self.server = server
        self.manipulate = manipulate
    
    def create_account(self, I, P):
        self.server.create_account(I, P) # Let's pretend I didn't see that.
    
    def authenticate(self, I, A, cb):
        print("Starting MITMing auth flow for", I)
        self.client = {'I': I, 'A': A, 'cb': cb}
        self.server.authenticate(I, A, self.authenticate_cb)
    
    def authenticate_cb(self, salt, B, session_id, cb):
        print("MITM server got server data back")
        self.server = {'salt': salt, 'B': B, 'session_id': session_id, 'cb': cb}
        if self.manipulate:
            print("Forcing b = 2, B = g^2")
            self.client['cb'](salt, pow(g, 2, N), session_id, self.validate_authentication)
        else:
            self.client['cb'](salt, B, session_id, self.validate_authentication)
    
    def validate_authentication(self, session_id, HMAC):
        print("MITM authentication validation intercepted.")
        if self.manipulate:
            uH = sha256(str(self.client['A']) + str(pow(g, 2, N)))
            u = bytes_to(uH, 'int')
            offline_dictionnary_attack(HMAC, self.server['salt'], self.client['A'],
                                       pow(g, 2, N), 2, u)
            print('Authentication not completed (because tampering). But we have the password !')
        self.server['cb'](session_id, HMAC)
        
import tqdm
def offline_dictionnary_attack(HMAC, salt, A, B, b, u):
    for word in tqdm.tqdm(words):
        xH = sha256(hex(salt) + word)
        x = bytes_to(xH, 'int')
        S_try = (pow(B, u*x, N)*pow(A, b, N)) % N
        K_try = bytes_to(sha256(str(S_try)),'base64')
        HMAC_try = sha256mac(K_try, str(salt))
        if HMAC_try == HMAC:
            print('Password found :', word)
            return word
    print('Password not found :(...')
    return None

In [21]:
Alice = Server()
Bob = Client('Bob', 'bob', 'bob@passw0rd')
Eve = MITMServer(Alice)

Bob.register_account(Eve)
Bob.authenticate(Eve)
print("")

# With password cracking
import random
Alice = Server()
Bob = Client('Bob', 'bob', random.choice(words))
Eve = MITMServer(Alice, manipulate=True)

Bob.register_account(Eve)
Bob.authenticate(Eve)

  2%|▏         | 20/1000 [00:00<00:04, 197.43it/s]

Bob is creating an account for username bob
Creating account for bob
Bob is trying to authenticate as bob
Starting MITMing auth flow for bob
Starting authentication flow for identity bob
MITM server got server data back
MITM authentication validation intercepted.
Client bob is authenticated !

Bob is creating an account for username bob
Creating account for bob
Bob is trying to authenticate as bob
Starting MITMing auth flow for bob
Starting authentication flow for identity bob
MITM server got server data back
Forcing b = 2, B = g^2
MITM authentication validation intercepted.


 68%|██████▊   | 677/1000 [00:02<00:01, 275.79it/s]

Password found : Andre's
Authentication not completed (because tampering). But we have the password !
Client bob failed authentication...


Given that Eve has intercepted the salt, Eve can deduce K from S. This means she has to crack S with the offline dictionnary attack. This works because :

$$\begin{align*}
S_{client}
&= B^{a+u.x} \ \ (\mathrm{mod}\ N) \\
&= B^{u.x}.B^{a} \ \ (\mathrm{mod}\ N) \\
&= B^{u.x}.A^{b} \ \ (\mathrm{mod}\ N) \\
\end{align*}$$

Eve has intercepted u and A, and has forced B and b, so it can simply bruteforce the value of K _offline_ !

### Exercice 39

Implement RSA

*Warning : uses an external lib for the primegen part, that wraps OpenSSL, as suggested*

In [22]:
# First, get some primitives down :

import gensafeprime

def primegen(small=True):
    """Generate a random prime."""
    if small:
        return gensafeprime.generate(32)
    return gensafeprime.generate(128)

def egcd(r, u, v, rp, up, vp):
    """Extended Euclid algorithm for GCD
    Given a, b, find GCD(a,b) and x,y such that :
        a.x + b.y = GCD(a,b)     (Bézout coefficients)
    """
    if r == 0:
        return up, vp
    else:
        return egcd(rp - (rp // r)*r, up - (rp // r)*u, vp - (rp // r)*v, r, u, v)
    
    
def invmod(a, N):
    """Calculate the inverse modulo"""
    x, _ = egcd(a, 1, 0, N, 0, 1)
    return x % N

assert (invmod(15, 37) * 15) % 37 == 1

In [126]:
import base64

def str_to_int(s):
    return int(base64.b16encode(s), 16)
def int_to_str(i):
    s = hex(i)[2:].upper()
    if len(s) % 2 == 1:
        s = '0' + s # base16 wants even length strings, adding a 0 in front does not change the value. Perfect.
    return base64.b16decode(s)
assert int_to_str(str_to_int((b"Hello, world !"))) == b'Hello, world !'

class RSA:
    def __init__(self, small=True):
        self.p = primegen(small=small)
        self.q = primegen(small=small)
        self.n = self.p * self.q
        self.et = (self.p - 1) * (self.q - 1)
        self.e = 3
        self.d = invmod(self.e, self.et)
        self.block_length = 3 + (not small)*9
    
    @property
    def public(self):
        return self.e, self.n
    
    @property
    def private(self):
        return self.d, self.n
    
    def encrypt_block(self, s, i2s = True):
        # Encrypt one block
        if isinstance(s, bytes):
            m = str_to_int(s)
            if m > self.n:
                print(m, self.n)
                raise Exception("Message not encryptable. Please use blocks.")
            if i2s:
                return int_to_str(pow(m, self.e, self.n))
            else:
                return pow(m, self.e, self.n)
        else:
            m = s
            return pow(m, self.e, self.n)
    
    def decrypt_block(self, s, i2s = True):
        if isinstance(s, bytes):
            c = str_to_int(s)
            if c > self.n:
                print(c, self.n)
                raise Exception("Message not encryptable. Please use blocks.")
            if i2s:
                return int_to_str(pow(c, self.d, self.n))
            else:
                return pow(c, self.d, self.n)
        else:
            c = s
            return pow(c, self.d, self.n)
    
    def encrypt(self, s):
        acc = []
        for i in range(0, len(s), self.block_length):
            acc.append(self.encrypt_block(s[i:i+self.block_length]))
        return acc
        
    
    def decrypt(self, s):
        return b"".join(self.decrypt_block(b) for b in s)
        
    
r = RSA(small=False)
assert r.decrypt(r.encrypt(b"Vanilla Ice Baby")) == b'Vanilla Ice Baby'

### Exercice 40

Break RSA with E = 3, broadcast attack.

In [127]:
r1, r2, r3 = RSA(small=False), RSA(small=False), RSA(small=False)
text = b'Cooking MC\'s like a pound of bacon'
c1, c2, c3 = r1.encrypt(text), r2.encrypt(text), r3.encrypt(text)
(_, n1),(_, n2),(_, n3) = r1.public, r2.public, r3.public

In [128]:
import decimal
def decrypt(c1, n1, c2, n2, c3, n3):
    c1, c2, c3 = str_to_int(c1), str_to_int(c2), str_to_int(c3)
    e1 = invmod(n2*n3, n1) * n2 * n3
    e2 = invmod(n1*n3, n2) * n1 * n3
    e3 = invmod(n1*n2, n3) * n1 * n2
    cleartext = (c1 * e1 + c2 * e2 + c3 * e3) % (n1 * n2 * n3)
    return round(pow(cleartext, decimal.Decimal(1)/3)) # Decimal allows to have for higher precision


c = b"".join([int_to_str(decrypt(c1i, n1, c2i, n2, c3i, n3)) for c1i, c2i, c3i in zip(c1, c2, c3)])
print(c) # It's not exaaaactly the original, so no assert here.
         # But very close and could be improved with a better cube root algorithm.

b'Cooking MC&\xe8 like a pou.d of bacon'
